<a href="https://colab.research.google.com/github/IshwarKapoor/IshwarKapoor/blob/main/Ch5_PPP_v1_Practical_data_science_in_python_book.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
import base64
import io # Import io module for BytesIO

#github details are below
owner="PacktPublishing"
repo="Practical-Data-Science-with-Python"
path="5-Chapter-5/data/PPP Data 150k plus 080820.zip"

api_url=f'https://api.github.com/repos/{owner}/{repo}/contents/{path}'


response=requests.get(api_url)
file_content=response.json() # Parse the JSON response to get file metadata

# Get the actual download URL for the zip file content from the metadata
download_url = file_content['download_url'] if 'download_url' in file_content else file_content['url']

# Download the actual zip file content
zip_response = requests.get(download_url)
zip_content = zip_response.content # Get the binary content


import zipfile
# Use io.BytesIO to treat the downloaded content as an in-memory file
with zipfile.ZipFile(io.BytesIO(zip_content), 'r') as zip_ref:
  zip_ref.extractall()

In [ ]:
zip_content

In [ ]:
import pandas as pd
import requests
import base64
import io # Import io module for BytesIO
import zipfile

#github details are below
owner="PacktPublishing"
repo="Practical-Data-Science-with-Python"
path="5-Chapter-5/data/PPP Data 150k plus 080820.zip"

api_url=f'https://api.github.com/repos/{owner}/{repo}/contents/{path}'

response=requests.get(api_url)
file_content=response.json() # Parse the JSON response to get file metadata

# Get the actual download URL for the zip file content from the metadata
download_url = file_content['download_url'] if 'download_url' in file_content else file_content['url']

# Download the actual zip file content
zip_response = requests.get(download_url)
zip_content = zip_response.content # Get the binary content

# Use io.BytesIO to treat the downloaded content as an in-memory file
with zipfile.ZipFile(io.BytesIO(zip_content), 'r') as zip_ref:
  # 1. How to tackle not knowing what files are in the zip:
  # Use namelist() to get a list of all files and directories in the archive
  print("Files in the zip archive:")
  file_list = zip_ref.namelist()
  for filename in file_list:
      print(f"- {filename}")

  # 2. Read them as a DataFrame directly without extracting to disk:
  # Assuming the CSV file is 'PPP Data 150k plus 080820.csv'
  csv_filename = "PPP Data 150k plus 080820.csv"
  if csv_filename in file_list:
      print(f"\nReading '{csv_filename}' directly into a DataFrame...")
      with zip_ref.open(csv_filename) as csv_file: # in this zip_ref file, open that csv file PPP and saved in csv_file obj
          # Read the content of the CSV file from the zip archive directly into a DataFrame
          df = pd.read_csv(csv_file)
          print("DataFrame head:")
          print(df.head())
  else:
      print(f"\n{csv_filename} not found in the zip archive. Cannot read into DataFrame directly.")

  # The original extractall() call, if still desired (commented out for direct reading example)
  # zip_ref.extractall()

In [ ]:
df.head()

In [ ]:
import pandas as pd
import requests
from io import BytesIO
import zipfile

#github details are below
owner="PacktPublishing"
repo="Practical-Data-Science-with-Python"
path="5-Chapter-5/data/PPP Data 150k plus 080820.zip"

api_url=f'https://api.github.com/repos/{owner}/{repo}/contents/{path}'


response=requests.get(api_url)
file_content=response.json()


zip_response=requests.get(file_content['download_url'])
zip_content=zip_response.content

with zipfile.ZipFile(BytesIO(zip_content), 'r') as zip_ref:
  print("success and "f'file name is {zip_ref.filelist[0].filename}')

  # FIX: To open a file from *within* the zip archive, use zip_ref.open()
  # The argument should be the actual filename string, obtained from zip_ref.filelist[0].filename
  with zip_ref.open(zip_ref.filelist[0].filename, 'r') as f:
    # If the content is text, you might need to decode it, e.g., f.read().decode('utf-8')
    # For this example, we'll just confirm it opened.
    print(f"\nSuccessfully opened '{zip_ref.filelist[0].filename}' from within the zip archive.")
    # You could now, for example, read its content:
    # first_line = f.readline().decode('utf-8').strip()
    # print(f"First line of the file: {first_line}")
    df=pd.read_csv(f)

df.head(10)

In [ ]:
zip_ref.filelist

In [ ]:
# Correct way to access the filename of the first item in the list:
zip_ref.filelist[0].filename

In [ ]:
type(zip_content)

In [ ]:
zip_content

In [ ]:
df.info() # 3 floats and rest objects

In [ ]:
df.shape # 662515 rows and 16 columns

In [ ]:
df.describe()
# missing a few items from each columns

In [ ]:
df['LoanRange'].value_counts()

In [ ]:
df.isna().sum()

In [ ]:
df['BusinessName'].nunique()

In [ ]:
import matplotlib.pyplot as plt

import seaborn as sns

import missingno as msno

msno.matrix(df)
plt.show()

In [ ]:
melted_df=df[['Zip', 'NAICSCode', 'JobsReported']].melt(var_name='Variable', value_name='Value')


sns.boxenplot(data=melted_df, x='Variable', y='Value')
plt.show()
# not very nice rep, lets plot sep figures

In [ ]:
plt.figure(figsize=(15,5))

plt.subplot(1,3,1)
sns.boxenplot(data=df, x='Zip')
plt.subplot(1,3,2)
sns.boxenplot(data=df, x='NAICSCode')
plt.subplot(1,3,3)
sns.boxenplot(data=df, x='JobsReported')

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(15,5))

plt.subplot(1,3,1)
sns.histplot(data=df, x='Zip', kde='True')
plt.subplot(1,3,2)
sns.histplot(data=df, x='NAICSCode', kde='True')
plt.subplot(1,3,3)
sns.histplot(data=df, x='JobsReported', kde='True')

plt.tight_layout()
plt.show()

# jobs reported is left skewed, rest has pockets of bimodal distribution

In [ ]:
plt.figure(figsize=(15,5))

plt.subplot(1,3,1)
sns.violinplot(data=df, x='Zip')
plt.subplot(1,3,2)
sns.violinplot(data=df, x='NAICSCode')
plt.subplot(1,3,3)
sns.violinplot(data=df, x='JobsReported')

plt.tight_layout()
plt.show()

In [ ]:
sns.pairplot(data=df)

In [ ]:
print(df['State'].nunique())
df['State'].unique()

In [ ]:
!pip install ydata-profiling

In [ ]:
from ydata_profiling import ProfileReport
report=ProfileReport(df)

In [ ]:
report

In [ ]:
# which states has the most jobs reported

#tp_5_stat=
#df.groupby('State').sum()

tp_5_stat=df.groupby('State').sum().sort_values(by='JobsReported', ascending=False)[0:5:1].index.values

# texas highest jobs reported

In [ ]:
tp_5_stat

In [ ]:
#df['DateApproved']=pd.to_datetime(df['DateApproved'], infer_datetime_format=True)
# no need for infer_datetime_format
df['DateApproved']=pd.to_datetime(df['DateApproved'])

In [ ]:
df.info()

In [ ]:
df['LoanRange'].describe()

In [ ]:
df['LoanRange'].value_counts(ascending=True)

In [ ]:
upper_bound={'a $5-10 million':10000000, 'b $2-5 million':5000000,'c $1-2 million':2000000, 'd $350,000-1 million':1000000,
             'e $150,000-350,000':350000}

lower_bound={'a $5-10 million':5000000, 'b $2-5 million':2000000,'c $1-2 million':1000000, 'd $350,000-1 million':350000,
             'e $150,000-350,000':150000}



In [ ]:
df['LoanRange'][55] in upper_bound
print(upper_bound[df['LoanRange'][55]])

# i can build loop to replace lower and upper bounds but
# would use a in built function

df['LBLoan'] = df['LoanRange'].replace(upper_bound)
df['UBLoan'] = df['LoanRange'].replace(lower_bound)

df.head(10)

In [ ]:
df.drop(columns=['UpperBounLoan', 'LowerBounLoan'], inplace=True)

In [ ]:
df.info()

In [ ]:
df['UBLoan'] = pd.to_numeric(df['UBLoan'])
df['LBLoan'] = pd.to_numeric(df['LBLoan'])

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df['LoanRange'].value_counts()

In [ ]:
df['LoanRange'].value_counts().index

In [ ]:
import seaborn as sns
sns.countplot(data=df, y='LoanRange', order=df['LoanRange'].value_counts().index)

In [ ]:
# next calc, loans vs jobsreported
# loan bounds vs states

#states vs total loans (can go with lower or upper)
